In [10]:
from lib import check_libs_installed, get_processing_folder_path

check_libs_installed()

All Required Installed


In [11]:
import pandas as pd
import numpy as np
from const import KB,MB,GB

In [12]:
df = pd.read_csv(get_processing_folder_path('processed_na.csv'))

In [13]:
df['Size Approximate'] = pd.NA
df.head(3)

,App Name,App Id,Category,Rating,Rating Count,Installs,Minimum Installs,Maximum Installs,Free,Price,Currency,Size,Minimum Android,Released,Last Updated,Content Rating,Ad Supported,In App Purchases,Editors Choice,Size Approximate
0,Gakondo,com.ishakwe.gakondo,Adventure,0.0,0.0,10+,10.0,15,True,0.0,USD,10M,7.1 and up,"Feb 26, 2020","Feb 26, 2020",Everyone,False,False,False,<NA>
1,Ampere Battery Info,com.webserveis.batteryinfo,Tools,4.4,64.0,"5,000+",5000.0,7662,True,0.0,USD,2.9M,5.0 and up,"May 21, 2020","May 06, 2021",Everyone,True,False,False,<NA>
2,Vibook,com.doantiepvien.crm,Productivity,0.0,0.0,50+,50.0,58,True,0.0,USD,3.7M,4.0.3 and up,"Aug 9, 2019","Aug 19, 2019",Everyone,False,False,False,<NA>


In [14]:
df['Size'].unique()

array(['10M', '2.9M', '3.7M', ..., '405M', '3.2k', '512M'], dtype=object)

In [15]:
df.loc[:,'Size'] = df['Size'].str.lower()

In [16]:
def size_to_number(s:pd.Series) -> pd.Series:
    if s['Size'].startswith('varies with device'):
        s['Size Approximate'] = 0 # for consistence
    else:
        unit = KB if s['Size'].endswith('k') else MB if s['Size'].endswith('m') else GB if s['Size'].endswith('g') else False
        if unit:
            n = ''.join(s['Size'][:-1].split(','))
            s['Size Approximate'] = int(unit * float(n))
    return s

In [17]:
df_size_to_float = df.apply(axis=1,func=size_to_number)

In [18]:
test = df_size_to_float[['Size','Size Approximate']]
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2290061 entries, 0 to 2290060
Data columns (total 2 columns):
 #   Column            Dtype 
---  ------            ----- 
 0   Size              object
 1   Size Approximate  int64 
dtypes: int64(1), object(1)
memory usage: 34.9+ MB


In [19]:
test['Size Approximate'].unique()

array([ 10485760,   3040870,   3879731, ..., 424673280,      3276,
       536870912], dtype=int64)

In [20]:
test[test['Size Approximate'].isna()]

,Size,Size Approximate


In [22]:
test['Size Approximate'].max()

1610612736

In [23]:
test[test['Size Approximate'] == 0].count()

Size                53048
Size Approximate    53048
dtype: int64

In [24]:
test['Size Approximate'].max()

# 1MB, 5MB, 50MB, 100MB, 200MB, 500MB, 1GB, 2GB, MORE THAN 2GB
bins = [
    -1,
    0,
    1*MB,
    5*MB,
    25*MB,
    50*MB,
    100*MB,
    200*MB,
    500*MB,
    1*GB,
    GB*GB
    ]

labels = [
    'Varies with device',
    '<= 1MB',
    '<= 5MB',
    '<= 25MB',
    '<= 50MB',
    '<= 100MB',
    '<= 200MB',
    '<= 500MB',
    '<= 1GB', 
    '> 1GB'
    ]

print(bins.__len__())
print(labels.__len__())

10
9


In [34]:
size_in_bin = pd.cut(test['Size Approximate'],bins=bins,labels=labels)
size_in_bin.name = 'Size Bin'

df_size_bind = pd.concat(objs=[df_size_to_float,size_in_bin],axis=1)

In [26]:
df['Category'].unique()

array(['Adventure', 'Tools', 'Productivity', 'Communication', 'Social',
       'Libraries & Demo', 'Lifestyle', 'Personalization', 'Racing',
       'Maps & Navigation', 'Travel & Local', 'Food & Drink',
       'Books & Reference', 'Medical', 'Puzzle', 'Entertainment',
       'Arcade', 'Auto & Vehicles', 'Photography', 'Health & Fitness',
       'Education', 'Shopping', 'Board', 'Music & Audio', 'Sports',
       'Beauty', 'Business', 'Educational', 'Finance', 'News & Magazines',
       'Casual', 'Art & Design', 'House & Home', 'Card', 'Events',
       'Trivia', 'Weather', 'Strategy', 'Word', 'Video Players & Editors',
       'Action', 'Simulation', 'Music', 'Dating', 'Role Playing',
       'Casino', 'Comics', 'Parenting'], dtype=object)

Game - Adventure, Racing, Puzzle, Arcade, Strategy, Action, Simulation
Tools - Tools, Productivity

In [32]:
df[df['Category'].str.match('Sports')]['App Name']

49                       Bowling Paradise Pro FREE
50                             ACtrainingLanzarote
93                               Pocket Bowling 3D
131                                        VfL1910
137                        Clube Atlético Ypiranga
                            ...                   
2289934    Dunk Basket : Basketball Dunk Hoop Game
2289944                    Spotlight Dance Academy
2289952                           H.K Golf Manager
2289955                                  La Molina
2290044                      Gear Ratio Calculator
Name: App Name, Length: 47052, dtype: object

In [35]:
df_size_bind['Size Bin'].value_counts()

<= 25MB     1117549
<= 5MB       539237
<= 50MB      345589
<= 100MB     168282
<= 1MB        38551
<= 200MB      26115
<= 500MB       1553
<= 1GB          127
> 1GB            10
Name: Size Bin, dtype: int64

In [36]:
df_size_bind

,App Name,App Id,Category,Rating,Rating Count,Installs,Minimum Installs,Maximum Installs,Free,Price,...,Size,Minimum Android,Released,Last Updated,Content Rating,Ad Supported,In App Purchases,Editors Choice,Size Approximate,Size Bin
0,Gakondo,com.ishakwe.gakondo,Adventure,0.0,0.0,10+,10.0,15,True,0.0,...,10m,7.1 and up,"Feb 26, 2020","Feb 26, 2020",Everyone,False,False,False,10485760,<= 25MB
1,Ampere Battery Info,com.webserveis.batteryinfo,Tools,4.4,64.0,"5,000+",5000.0,7662,True,0.0,...,2.9m,5.0 and up,"May 21, 2020","May 06, 2021",Everyone,True,False,False,3040870,<= 5MB
2,Vibook,com.doantiepvien.crm,Productivity,0.0,0.0,50+,50.0,58,True,0.0,...,3.7m,4.0.3 and up,"Aug 9, 2019","Aug 19, 2019",Everyone,False,False,False,3879731,<= 5MB
3,Smart City Trichy Public Service Vehicles 17UC...,cst.stJoseph.ug17ucs548,Communication,5.0,5.0,10+,10.0,19,True,0.0,...,1.8m,4.0.3 and up,"Sep 10, 2018","Oct 13, 2018",Everyone,True,False,False,1887436,<= 5MB
4,GROW.me,com.horodyski.grower,Tools,0.0,0.0,100+,100.0,478,True,0.0,...,6.2m,4.1 and up,"Feb 21, 2020","Nov 12, 2018",Everyone,False,False,False,6501171,<= 25MB
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2290056,大俠客—熱血歸來,com.rxsj.ssjj,Role Playing,4.3,16775.0,"100,000+",100000.0,337109,True,0.0,...,77m,4.1 and up,NaN,"Jun 01, 2021",Teen,False,False,False,80740352,<= 100MB
2290057,ORU Online,com.threedream.oruonline,Education,0.0,0.0,100+,100.0,430,True,0.0,...,44m,4.1 and up,"Jan 17, 2018","Feb 02, 2018",Everyone,False,False,False,46137344,<= 50MB
2290058,Data Structure,datastructure.appoworld.datastucture,Education,0.0,0.0,100+,100.0,202,True,0.0,...,29m,5.0 and up,"Aug 19, 2018","Aug 19, 2018",Everyone,False,False,False,30408704,<= 50MB
2290059,Devi Suktam,ishan.devi.suktam,Music & Audio,3.5,8.0,"1,000+",1000.0,2635,True,0.0,...,10m,5.0 and up,"Aug 1, 2016","May 05, 2021",Everyone,True,False,False,10485760,<= 25MB


In [37]:
df_size_bind.info()
df_size_bind.dropna()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2290061 entries, 0 to 2290060
Data columns (total 21 columns):
 #   Column            Dtype   
---  ------            -----   
 0   App Name          object  
 1   App Id            object  
 2   Category          object  
 3   Rating            float64 
 4   Rating Count      float64 
 5   Installs          object  
 6   Minimum Installs  float64 
 7   Maximum Installs  int64   
 8   Free              bool    
 9   Price             float64 
 10  Currency          object  
 11  Size              object  
 12  Minimum Android   object  
 13  Released          object  
 14  Last Updated      object  
 15  Content Rating    object  
 16  Ad Supported      bool    
 17  In App Purchases  bool    
 18  Editors Choice    bool    
 19  Size Approximate  int64   
 20  Size Bin          category
dtypes: bool(4), category(1), float64(4), int64(2), object(10)
memory usage: 290.5+ MB


In [38]:
df_size_bind.to_csv(get_processing_folder_path('processed_temp.csv'),index=False)

In [39]:
df_size_bind.isna().sum()

App Name                0
App Id                  0
Category                0
Rating                  0
Rating Count            0
Installs                0
Minimum Installs        0
Maximum Installs        0
Free                    0
Price                   0
Currency               20
Size                    0
Minimum Android      6526
Released            48378
Last Updated            0
Content Rating          0
Ad Supported            0
In App Purchases        0
Editors Choice          0
Size Approximate        0
Size Bin            53048
dtype: int64